- Just run all of it quickly when there are new subject to normalize the data and concatenate
- By normalization here I mean to per voxel across its time series.

# Import stuff

In [1]:
import os
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas as pd

# Params

In [2]:
#dirs:
pre_proc_dir = 'data/pre_proc_data'
rs_data_path_IDCH_sub_Nums = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums'
rs_data_path_IDCH_sub_Nums_normed = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums_normed' # Here each voxel time series will be normed 
rs_data_path_IDCH_sub_Nums_normed_concat = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums_normed_concat' # Here each voxel time series will be normed and concatenated

# file_format:
any_session_file_format = '_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'

# R01 to IDCH mapping:
mapping_R01_to_IDCH = {'222': '101', '183': '102', '216': '103', '192': '104', '251': '105', '206': '106', '180': '107', '184': '108', '169': '109', '207': '110',
                       '159': '111', '115': '112', '114': '113', '232': '114', '173': '115', '171': '117', '215': '118', '265': '119', '177': '120', '269': '121',
                       '261': '122'}


## Copy data to be with IDCH sub numbers (and extract the brain)

In [3]:
# get all directories in the pre_proc_dir:
r01_sub_dirs = [f for f in os.listdir(pre_proc_dir) if os.path.isdir(os.path.join(pre_proc_dir, f))]
# iterate over all subjects:
for dir in r01_sub_dirs:
    sub_dir = os.path.join(pre_proc_dir, dir, 'func') 
    sub_R01_id = dir.split('Hab0')[1][:3] # get sub R01 id
    sub_IDCH_id = mapping_R01_to_IDCH[sub_R01_id] # get sub IDCH id

    # get subject's resting state:
    rs_files = [f for f in os.listdir(sub_dir) if any_session_file_format in f]
    for f in rs_files:
        # get the session number:
        session = f.split('rest_run-')[1][0]
        new_file_name = f'sub-{sub_IDCH_id}_ses-{session}_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz'
        # copy the file if not exists:
        if not os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)):
            print(f'> Copying {f} to {new_file_name}')
            # os.system(f'cp {os.path.join(sub_dir, f)} {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
            os.system(f'fslmaths {os.path.join(sub_dir, f)} -mas $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
        # write the file name to the input_files.txt file:


# Normalized the data (per voxel across its time series)

In [4]:
# If normed folder does not exist, create it:
if not os.path.exists(rs_data_path_IDCH_sub_Nums_normed):
    os.makedirs(rs_data_path_IDCH_sub_Nums_normed)

# iterate over all subjects:
file_list = os.listdir(rs_data_path_IDCH_sub_Nums)
file_list.sort()
for file in file_list:
    subject = file.split('_')[0].split('-')[1]
    session = file.split('ses-')[1][0]
    ts_normed_file_name = f'sub-{subject}_ses-{session}_rest_MNI152-smoothAROMAnonaggr_bold_brain_normed.nii.gz'
    if not os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums_normed, ts_normed_file_name)):
        print(f'> Create a normalized version (per voxel across the time series) for {subject} session {session}')
        # calculate the mean and std for each voxel:
        temp_mean_file = os.path.join(rs_data_path_IDCH_sub_Nums_normed, f'{file.split(".")[0]}_mean.nii.gz')
        temp_std_file = os.path.join(rs_data_path_IDCH_sub_Nums_normed, f'{file.split(".")[0]}_std.nii.gz')
        os.system(f'fslmaths {os.path.join(rs_data_path_IDCH_sub_Nums, file)} -Tmean {temp_mean_file}')
        os.system(f'fslmaths {os.path.join(rs_data_path_IDCH_sub_Nums, file)} -Tstd {temp_std_file}')
        # normalize the time series:
        os.system(f'fslmaths {os.path.join(rs_data_path_IDCH_sub_Nums, file)} -sub {temp_mean_file} -div {temp_std_file} {os.path.join(rs_data_path_IDCH_sub_Nums_normed, ts_normed_file_name)}')
        # remove the temp files:
        os.system(f'rm {temp_mean_file}')
        os.system(f'rm {temp_std_file}')


# Concatenate the normed ts data from the 2 sessions into one time series

In [5]:
# if normed_concat folder does not exist, create it:
if not os.path.exists(rs_data_path_IDCH_sub_Nums_normed_concat):
    os.makedirs(rs_data_path_IDCH_sub_Nums_normed_concat)

# get all directories in the root directory
normed_sub_list = os.listdir(rs_data_path_IDCH_sub_Nums_normed)
# get the unique subjects:
normed_sub_list = list(set([x.split('_')[0] for x in normed_sub_list]))
normed_sub_list.sort()

for subj in normed_sub_list:
    # get first rest session
    first_rest_session_file = [f for f in os.listdir(rs_data_path_IDCH_sub_Nums_normed) if f'{subj}_ses-1' in f][0]
    # get second rest session
    second_rest_session_file = [f for f in os.listdir(rs_data_path_IDCH_sub_Nums_normed) if f'{subj}_ses-2' in f][0]

    # concatenate session using fslmerge:
    # fslmerge -t <output_file> <input_files>
    if os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums_normed_concat, f"{subj}_IDCH_id_rs_normed_concat.nii.gz")):
        continue
    print(f'merging data for {subj}')
    os.system(f'fslmerge -t {os.path.join(rs_data_path_IDCH_sub_Nums_normed_concat, f"{subj}_IDCH_id_rs_normed_concat.nii.gz")} {os.path.join(rs_data_path_IDCH_sub_Nums_normed, first_rest_session_file)} {os.path.join(rs_data_path_IDCH_sub_Nums_normed, second_rest_session_file)}')


merging data for sub-122
